# Is there a correlation between maximum temperatures increasing and a larger diurnal range in Melbourne?

## Introduction

This analysis looks at the Melbourne minimum and maximum temperatures observed in Melbourne, Australia to see if there is any correlation between maximum temperatures increasing and a larger range of temperatures experienced over the day (diurnal range).

High diurnal range climates tend to put more pressure on power generation and transport infrastructure as well as civil engineering considerations. Building design that uses high thermal mass wall cladding becomes more relevant the greater the range of temperature experienced. (Ranges of 7 to 10 degrees C, thermal mass materials can be useful, depending on climate; diurnal ranges over 10 degrees C high mass construction is desirable). The health of vulnerable members of the population, flora and fauna may also be negatively impacted by a broader diurnal temperature range.

## Data sources

The ACORN-SAT daily minimum temperature and daily maximum temperature datasets were used, with full years data between 1910 and 2014 at the Melbourne Regional OFfice.

* http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.minT.086071.daily.txt
* http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.maxT.086071.daily.txt

## References

* [Spearmans correlation and classification of strength of correlation](http://www.statstutor.ac.uk/resources/uploaded/spearmans.pdf)
* [Building Schools in the 21st Century](https://www.eduweb.vic.gov.au/edulibrary/public/publ/research/publ/Building_schools_in_the_21st_century.pdf) - specifically p.8 mentions thermal mass and diurnal range.
* [Impacts and Adaption Responses of infrastructure and communities during heatwaves](https://www.nccarf.edu.au/business/sites/www.nccarf.edu.au.business/files/attached_files_publications/Pub%2013_10%20Southern%20Cities%20Heatwaves%20-%20Complete%20Findings.pdf)
* [Urban Infrastructure: Finance and Management](https://books.google.com.au/books?id=XfRyUH4771wC&pg=PT228&lpg=PT228&dq=diurnal+range+effect+on+infrastructure&source=bl&ots=cQyCmejkVw&sig=kkDcINlmccefEQJvPivzCUkKibk&hl=en&sa=X&ved=0ahUKEwivhfy0ranJAhUkOKYKHc5FCIoQ6AEIQzAJ#v=onepage&q=diurnal%20range%20effect%20on%20infrastructure&f=false)

## Processing the data

### Loading the data

In [54]:
URL_MIN = "http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.minT.086071.daily.txt"
URL_MAX = "http://www.bom.gov.au/climate/change/acorn/sat/data/acorn.sat.maxT.086071.daily.txt"
DATE_HEADING = "date"
MIN_T_HEADING = "minT"
MAX_T_HEADING = "maxT"

In [43]:
from pandas import *

In [44]:
## load minT data
minT = read_table(
    URL_MIN, 
    header=None, 
    sep="\s*", # some amount of whitespace
    skiprows=1, 
    engine="python", # required to use regex in sep
    index_col=False,
    na_values=99999.9,
    names=[DATE_HEADING,MIN_T_HEADING]) # no header supplied
## check dtypes (as dtypes not supported with engine python)
minT.dtypes

date      int64
minT    float64
dtype: object

In [45]:
# check head and tail
minT.head()

,date,minT
0,19100101,9.9
1,19100102,14.5
2,19100103,24.3
3,19100104,17.7
4,19100105,13.8


In [46]:
minT.tail()

,date,minT
38346,20141227,12.2
38347,20141228,13.4
38348,20141229,17.5
38349,20141230,13.4
38350,20141231,11.9


In [47]:
## load maxT data
maxT = read_table(
    URL_MAX, 
    header=None, 
    sep="\s*", # some amount of whitespace
    skiprows=1, 
    engine="python", # required to use regex in sep
    index_col=False,
    na_values=99999.9,
    names=[DATE_HEADING,MAX_T_HEADING]) # no header supplied
## check dtypes (as dtypes not supported with engine python)
minT.dtypes

date      int64
minT    float64
dtype: object

In [48]:
# check head and tail
maxT.head()

,date,maxT
0,19100101,30.7
1,19100102,38.6
2,19100103,36.9
3,19100104,23.3
4,19100105,28.5


In [49]:
maxT.tail()

,date,maxT
38346,20141227,23.0
38347,20141228,32.1
38348,20141229,26.4
38349,20141230,24.6
38350,20141231,25.8


### Merging minimum and maximum datasets into a single dataframe

In [60]:
data = merge(minT, maxT, on=DATE_HEADING, how="inner")
data.head()

,date,minT,maxT
0,19100101,9.9,30.7
1,19100102,14.5,38.6
2,19100103,24.3,36.9
3,19100104,17.7,23.3
4,19100105,13.8,28.5


### Changing date to a datetime dtype and using it as the index

In [61]:
data[DATE_HEADING] = to_datetime(data[DATE_HEADING],format="%Y%m%d")
data.head()

,date,minT,maxT
0,1910-01-01,9.9,30.7
1,1910-01-02,14.5,38.6
2,1910-01-03,24.3,36.9
3,1910-01-04,17.7,23.3
4,1910-01-05,13.8,28.5


In [62]:
data.index = data['date']
data.head()

,date,minT,maxT
date,,,
1910-01-01,1910-01-01,9.9,30.7
1910-01-02,1910-01-02,14.5,38.6
1910-01-03,1910-01-03,24.3,36.9
1910-01-04,1910-01-04,17.7,23.3
1910-01-05,1910-01-05,13.8,28.5


In [71]:
HEADINGS = [MIN_T_HEADING, MAX_T_HEADING]
data = data[HEADINGS]
data.head()

,minT,maxT
date,,
1910-01-01,9.9,30.7
1910-01-02,14.5,38.6
1910-01-03,24.3,36.9
1910-01-04,17.7,23.3
1910-01-05,13.8,28.5


In [93]:
#%matplotlib inline
#data.plot(figsize=(15,5))

### Adding range calculations to the dataframe

In [83]:
DIURNAL_RANGE_HEADING = 'Range'
data[DIURNAL_RANGE_HEADING] = data[MAX_T_HEADING] - data[MIN_T_HEADING]
data.head()

,minT,maxT,Range
date,,,
1910-01-01,9.9,30.7,20.8
1910-01-02,14.5,38.6,24.1
1910-01-03,24.3,36.9,12.6
1910-01-04,17.7,23.3,5.6
1910-01-05,13.8,28.5,14.7


In [95]:
data.to_csv("Melbourne_Climate.csv", index_label='date')

In [86]:
range_data = data[DIURNAL_RANGE_HEADING]
range_data.head()

date
1910-01-01    20.8
1910-01-02    24.1
1910-01-03    12.6
1910-01-04     5.6
1910-01-05    14.7
Name: Range, dtype: float64

In [94]:
#range_data.plot()

### Analysing maximum temperature and diurnal range for correlation

In [91]:
from scipy.stats import spearmanr
maxColumn = data[MAX_T_HEADING]
rangeColumn = data[DIURNAL_RANGE_HEADING]
(correlation, pValue) = spearmanr(maxColumn,rangeColumn)
def analyseCorrelation(correlation, pValue):
    print('The correlation is', correlation)
    posTest = correlation
    if correlation < 0: posTest = correlation * -1
    if posTest < 0.19: print('The correlation is very weak.')
    elif posTest < 0.39: print('The correlation is weak.')
    elif posTest < 0.59: print('The correlation is moderate.')
    elif posTest < 0.79: print('The correlation is strong.')
    else: print('The correlation is very strong.')
    if pValue < 0.05:
        print('It is statistically significant.')
    else:
        print('It is not statistically significant.')
analyseCorrelation(correlation, pValue)

The correlation is 0.654355320973
The correlation is strong.
It is statistically significant.


### Analysing minimum temperature and diurnal range for correlation

In [92]:
minColumn = data[MIN_T_HEADING]
(correlation, pValue) = spearmanr(minColumn,rangeColumn)
analyseCorrelation(correlation, pValue)

The correlation is -0.0086541595709
The correlation is very weak.
It is not statistically significant.


## Conclusions

There is a statistically significant, strong correlation between **maximum temperatures** and **diurnal range**, that is, a strong and significant correlation when maximum temperatures go up, so does diurnal range.

However, looking at the minimum temperature and diurnal range, there is a not statistically significant correlation and the inverse correlation is very weak.

This means that warmer maximum temperatures are likely to see a larger diurnal range, which may have negative effects on infrastructure including transport, power and buildings, as well as the health of vulnerable people, plants and animals. In climates where high thermal mass construction works well, and with higher maximum temperatures expected due to climate change, high thermal mass building design seems like something worth looking into because thermal comfort is something that will also impact upon power use as well as health of vulnerable groups.